In [70]:
import os
import pprint
import numpy as np
import pandas as pd
import cv2
import shutil

In [71]:
def findMultiTumour(csv_path, abnormality_col):
    try:
        df = pd.read_csv(csv_path, header=0)

        multi_df = df.loc[df[abnormality_col] > 1]

        multi_tumour_list = []
        for row in multi_df.itertuples():

            patient_id = row.patient_id
            lr = row.left_or_right_breast
            img_view = row.image_view

            identifier = "_".join([patient_id, lr, img_view])
            multi_tumour_list.append(identifier)

        multi_tumour_set = set(multi_tumour_list)

    except Exception as e:
        print((f"Unable to findMultiTumour!\n{e}"))

    return multi_tumour_set

In [72]:
def masksToSum(img_path, multi_tumour_set, extension):

    try:
        images = [
            f
            for f in os.listdir(img_path)
            if (not f.startswith(".") and f.endswith(extension))
        ]

        masks_to_sum = [
            m
            for m in images
            if ("MASK" in m and any(multi in m for multi in multi_tumour_set))
        ]

        masks_to_sum_dict = {patient_id: [] for patient_id in multi_tumour_set}

        for k, _ in masks_to_sum_dict.items():
            v = [os.path.join(img_path, m) for m in masks_to_sum if k in m]
            masks_to_sum_dict[k] = sorted(v)

        to_pop = [k for k, v in masks_to_sum_dict.items() if len(v) == 1]

        for k in to_pop:
            masks_to_sum_dict.pop(k)

    except Exception as e:
        print((f"Unable to get findMultiTumour!\n{e}"))

    return masks_to_sum_dict

In [73]:
def sumMasks(mask_list):

    summed_mask = np.zeros(mask_list[0].shape)

    for arr in mask_list:
        summed_mask = np.add(summed_mask, arr)

    _, summed_mask_bw = cv2.threshold(
        src=summed_mask, thresh=1, maxval=255, type=cv2.THRESH_BINARY
    )

    return summed_mask_bw

In [74]:
config_mmt = {
    "paths": {
        "images": "data\\Calc\\Test\\ALL PNG",
        "csv": "data\\Calc\\Updated_Calc_Test.csv",
        "output": "data\\Calc\\Test\\Summed MASKS"
    },
    "abnormality_col": "abnormality_id",
    "extension": ".png",
    "save": True
}

In [75]:
img_path = config_mmt["paths"]["images"]
csv_path = config_mmt["paths"]["csv"]
abnormality_col = config_mmt["abnormality_col"]
extension = config_mmt["extension"]
output_path = config_mmt["paths"]["output"]
save = config_mmt["save"]

In [76]:
multi_tumour_set = findMultiTumour(
        csv_path=csv_path, abnormality_col=abnormality_col
    )

In [77]:
masks_to_sum_dict = masksToSum(
        img_path=img_path, multi_tumour_set=multi_tumour_set, extension=extension
    )

In [79]:
sum_list = []
for k, v in masks_to_sum_dict.items():

        mask_list = [cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE) for mask_path in v]
        
        print("summing", k, "masks")
        
        sum_list.append(k)
        
        reference_shape = mask_list[0].shape
        for i in range(len(mask_list)):
            mask = mask_list[i]
            reshaped_mask = cv2.resize(mask, reference_shape[::-1], interpolation=cv2.INTER_NEAREST)
            mask_list[i] = reshaped_mask
            
        summed_mask = sumMasks(mask_list=mask_list)

        if save:
            if "train" in img_path.lower():
                save_path = os.path.join(
                    output_path, ("_".join(["Calc-Training", k, "MASK___PRE.png"]))
                )
            elif "test" in img_path.lower():
                save_path = os.path.join(
                    output_path, ("_".join(["Calc-Test", k, "MASK___PRE.png"]))
                )
            cv2.imwrite(save_path, summed_mask)


summing P_00140_LEFT_MLO masks
summing P_01538_RIGHT_MLO masks
summing P_00214_LEFT_MLO masks
summing P_01211_RIGHT_CC masks
summing P_01390_RIGHT_CC masks
summing P_00906_LEFT_MLO masks
summing P_00562_LEFT_CC masks
summing P_01211_RIGHT_MLO masks
summing P_01867_LEFT_MLO masks
summing P_00562_LEFT_MLO masks
summing P_01867_LEFT_CC masks
summing P_00495_RIGHT_CC masks
summing P_00140_RIGHT_CC masks
summing P_01272_LEFT_MLO masks
summing P_01845_RIGHT_MLO masks
summing P_00723_LEFT_MLO masks
summing P_00495_RIGHT_MLO masks
summing P_00353_LEFT_MLO masks
summing P_00077_RIGHT_CC masks
summing P_01460_LEFT_MLO masks
summing P_01845_RIGHT_CC masks
summing P_00038_RIGHT_CC masks
summing P_01670_LEFT_MLO masks
summing P_01460_RIGHT_MLO masks
summing P_00077_RIGHT_MLO masks
summing P_01030_RIGHT_CC masks
summing P_01272_LEFT_CC masks
summing P_00140_LEFT_CC masks
summing P_01004_LEFT_MLO masks
summing P_00038_RIGHT_MLO masks
summing P_00353_LEFT_CC masks
summing P_01004_LEFT_CC masks
summing

In [82]:
destination_folder = "data\\Calc\\Test\\ALL MASKS"
files = os.listdir("data\\Calc\\Test\\MASK")

for file in files:

    file_name = os.path.basename(file)
    split_parts = file_name.split("_")
    s = split_parts[1]+"_"+split_parts[2]+"_"+split_parts[3]+"_"+split_parts[4]
    if s not in sum_list:
        source_file = "data\\Calc\\Test\\MASK\\"+file_name
        shutil.copy(source_file, destination_folder)
        print(file_name)

Calc-Test_P_00038_LEFT_CC_MASK_1___PRE.png
Calc-Test_P_00038_LEFT_MLO_MASK_1___PRE.png
Calc-Test_P_00041_LEFT_CC_MASK_2___PRE.png
Calc-Test_P_00041_LEFT_MLO_MASK_2___PRE.png
Calc-Test_P_00077_LEFT_CC_MASK_1___PRE.png
Calc-Test_P_00077_LEFT_MLO_MASK_1___PRE.png
Calc-Test_P_00100_RIGHT_CC_MASK_1___PRE.png
Calc-Test_P_00100_RIGHT_MLO_MASK_1___PRE.png
Calc-Test_P_00127_RIGHT_CC_MASK_1___PRE.png
Calc-Test_P_00127_RIGHT_MLO_MASK_1___PRE.png
Calc-Test_P_00132_LEFT_MLO_MASK_1___PRE.png
Calc-Test_P_00141_LEFT_CC_MASK_1___PRE.png
Calc-Test_P_00141_LEFT_MLO_MASK_1___PRE.png
Calc-Test_P_00150_RIGHT_MLO_MASK_1___PRE.png
Calc-Test_P_00163_LEFT_CC_MASK_1___PRE.png
Calc-Test_P_00163_LEFT_MLO_MASK_1___PRE.png
Calc-Test_P_00164_RIGHT_CC_MASK_1___PRE.png
Calc-Test_P_00180_LEFT_CC_MASK_1___PRE.png
Calc-Test_P_00180_LEFT_MLO_MASK_1___PRE.png
Calc-Test_P_00195_LEFT_CC_MASK_1___PRE.png
Calc-Test_P_00195_LEFT_MLO_MASK_1___PRE.png
Calc-Test_P_00202_RIGHT_CC_MASK_2___PRE.png
Calc-Test_P_00202_RIGHT_MLO_MASK_2__

Calc-Test_P_01752_LEFT_CC_MASK_1___PRE.png
Calc-Test_P_01752_LEFT_MLO_MASK_1___PRE.png
Calc-Test_P_01773_RIGHT_CC_MASK_1___PRE.png
Calc-Test_P_01773_RIGHT_MLO_MASK_1___PRE.png
Calc-Test_P_01803_RIGHT_CC_MASK_1___PRE.png
Calc-Test_P_01803_RIGHT_MLO_MASK_1___PRE.png
Calc-Test_P_01820_LEFT_MLO_MASK_1___PRE.png
Calc-Test_P_01834_LEFT_MLO_MASK_1___PRE.png
Calc-Test_P_01842_LEFT_MLO_MASK_1___PRE.png
Calc-Test_P_01861_LEFT_CC_MASK_1___PRE.png
Calc-Test_P_01861_LEFT_MLO_MASK_1___PRE.png
Calc-Test_P_01868_LEFT_CC_MASK_1___PRE.png
Calc-Test_P_01868_LEFT_MLO_MASK_1___PRE.png
Calc-Test_P_01872_LEFT_CC_MASK_1___PRE.png
Calc-Test_P_01872_LEFT_MLO_MASK_1___PRE.png
Calc-Test_P_01882_LEFT_CC_MASK_1___PRE.png
Calc-Test_P_01882_LEFT_MLO_MASK_1___PRE.png
Calc-Test_P_01883_RIGHT_CC_MASK_1___PRE.png
Calc-Test_P_01883_RIGHT_MLO_MASK_1___PRE.png
Calc-Test_P_02102_LEFT_CC_MASK_1___PRE.png
Calc-Test_P_02102_LEFT_MLO_MASK_1___PRE.png
Calc-Test_P_02139_LEFT_CC_MASK_1___PRE.png
Calc-Test_P_02139_LEFT_MLO_MASK_1___